# DQM Maximization of COVID Vaccine Distribution
We applied a DQM solution of the Knapsack problem to the issue of COVID-19 vaccine distribution. Specifically, we focused on how vaccines arriving in the United States would be distributed to individual states. 

In this model, the federal vaccine distribution center is receiving offers from states to ship a given number of vaccines. The distribution center's goal is to maximize the number of people who receive the vaccine. It must consider the rate of vaccine administration per state as well as the sizes of the shipment offers to put as many vaccines as possible to good use.

Let' start by importing the tools we need: a shipment mocker to generate some shipment offers, an shipment analyzer to estimate how many administrations will occur, and a knapsack solver to find the optimal solution.

In [1]:
from shipment_mocking import create_mock_shipment
from shipment_analysis import estimate_shipment_value
from knapsack_solver import KnapsackSolver

Next, let's set up some constants for this scenario. Since we aren't in charge of real shipments, we need to generate a number of fake shipment offers and imagine we have a supply of vaccines with which to fulfill them. Since we're dealing with large numbers here, the scaling factor helps make the numbers a bit nicer for computation.

In [2]:
NUM_SHIPMENTS = 10
NUM_VACCINE_DOSES = 200_000
SCALING_FACTOR = 1.

It's time to create the shipment offers. These are the items of the knapsack problem. The weight is the number of doses to be shipped and the value is the number of doses expected to be administered.

In [3]:
shipments = []

for _ in range(NUM_SHIPMENTS):
    shipment = create_mock_shipment()
    weight = shipment["doses"]
    value = estimate_shipment_value(shipment)

    print(f"Shipment: {shipment}, value: {value}")
    shipments.append((shipment, weight, value))

Shipment: {'state': 'CA', 'vaccine_type': 'Moderna Kit   ', 'doses': 2000}, value: 1172
Shipment: {'state': 'MA', 'vaccine_type': 'Moderna Pallet', 'doses': 43200}, value: 22277
Shipment: {'state': 'CA', 'vaccine_type': 'Moderna Pallet', 'doses': 43200}, value: 25326
Shipment: {'state': 'MA', 'vaccine_type': 'Moderna Kit   ', 'doses': 1000}, value: 515
Shipment: {'state': 'OH', 'vaccine_type': 'Moderna Pallet', 'doses': 72000}, value: 39338
Shipment: {'state': 'OH', 'vaccine_type': 'Moderna Pallet', 'doses': 68400}, value: 37371
Shipment: {'state': 'CA', 'vaccine_type': 'Moderna Pallet', 'doses': 68400}, value: 40100
Shipment: {'state': 'CA', 'vaccine_type': 'Moderna Kit   ', 'doses': 1100}, value: 644
Shipment: {'state': 'MA', 'vaccine_type': 'Moderna Pallet', 'doses': 72000}, value: 37128
Shipment: {'state': 'CA', 'vaccine_type': 'Moderna Pallet', 'doses': 54000}, value: 31658


Now we have everything we need to set up the knapsack problem. Our solver makes it easy; simply create a new solver and add each item with its weight and value.

In [4]:
knapsack = KnapsackSolver(1.)

for shipment, weight, value in shipments:
    knapsack.add_item(shipment, weight / SCALING_FACTOR, value / SCALING_FACTOR)

To run the DQM solver, you need to configure access to D-Wave's online solvers.  You can create a configuration file using the interactive CLI as described [here](https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html#creating-a-configuration-file).

If you want to see the details of the DQM before submitting it to the solver, you can change the last argument to `debug=True`. 

In [5]:
print("Solving...", end="")
item_list, total_value, total_weight = knapsack.solve(NUM_VACCINE_DOSES / SCALING_FACTOR, debug=True)
print(" done.")

Solving...[ 0.00000000e+00 -1.09440926e+15]
[ 0.00000000e+00 -2.11921628e+16]
[ 0.00000000e+00 -2.11921628e+16]
[ 0.00000000e+00 -5.48579514e+14]
[ 0.00000000e+00 -3.24693078e+16]
[ 0.00000000e+00 -3.11843943e+16]
[ 0.00000000e+00 -3.11843943e+16]
[ 0.00000000e+00 -6.03286228e+14]
[ 0.00000000e+00 -3.24693078e+16]
[ 0.000000e+00 -2.568837e+16]
[ 0.00000000e+00 -5.49953025e+11]
[ 0.0000000e+00 -1.0999033e+12]
[ 0.0000000e+00 -2.1997956e+12]
[ 0.00000000e+00 -4.39954721e+12]
[ 0.00000000e+00 -8.79891843e+12]
[ 0.00000000e+00 -1.75971329e+13]
[ 0.00000000e+00 -3.51914501e+13]
[ 0.00000000e+00 -7.03716371e+13]
[ 0.00000000e+00 -1.40698222e+14]
[ 0.00000000e+00 -2.81216235e+14]
[ 0.00000000e+00 -5.61711633e+14]
[ 0.00000000e+00 -1.12053992e+15]
[ 0.00000000e+00 -2.22954646e+15]
[ 0.00000000e+00 -4.41295941e+15]
[ 0.00000000e+00 -8.64138474e+15]
[ 0.00000000e+00 -1.65446332e+16]
[ 0.00000000e+00 -3.01367212e+16]
[ 0.00000000e+00 -4.84632615e+16]
0 1 {(1, 1): 237580300800000.0}
0 2 {(1, 1): 2

Let's take a look at the final solution:

In [6]:
total_doses = 0
for item, quantity in item_list:
    print(f"{quantity} shipment{'s' if quantity != 1 else ''} of {item['doses']} doses delivered to {item['state']}")
    total_doses += item["doses"]

print(f"\nOut of {NUM_VACCINE_DOSES} doses available, {int(total_weight * SCALING_FACTOR)} delivered, {total_value} administered")

1 shipment of 2000 doses delivered to CA
1 shipment of 1000 doses delivered to MA

Out of 200000 doses available, 3001 delivered, -1570208.0 administered
